<a href="https://colab.research.google.com/github/olexandr7/erm_visitation_analysis/blob/main/ERM_tickets_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

General idea for now is quite straightforward - we clean up data a bit and then export to parquet files; parquet files get loaded to warehouse in Snowflake;
could also do direct Python - Snowflake pipeline but for MVP it should be fine  the way it is;

In [125]:
import pandas as pd

In [160]:
# Step 1: Read log file
df = pd.read_csv(
    'card.log.0',
    delimiter=' ',
    names=["date", "timezone", "device_id", "ticket_info", "ticket_id"],
    engine='python'  # safer for odd delimiters
)

In [ ]:
# Step 2: Clean and combine timestamp strings
df['timestamp_str'] = df['date'].str.strip('[') + ' ' + df['timezone'].str.strip(']')

# Step 3: Parse into timezone-aware datetime
df['timestamp'] = pd.to_datetime(
    df['timestamp_str'],
    format='%d/%b/%Y:%H:%M:%S %z',
    errors='coerce'  # safely handle bad rows
)

# Step 4: Drop temporary columns (optional)
df = df.drop(columns=['date', 'timezone', 'timestamp_str'])

In [163]:
# Parse into two separate dataframes based on timezone
df_utc3 = df[df['timestamp'].astype(str).str.contains(r'\+03:00$', regex=True)].copy()
df_utc2 = df[df['timestamp'].astype(str).str.contains(r'\+02:00$', regex=True)].copy()

In [166]:
# Doing timezone-specific conversions
df_utc2['timestamp'] = pd.to_datetime(df_utc2['timestamp'], errors='coerce')
df_utc2['timestamp'] = df_utc2['timestamp'].dt.tz_localize(None)

df_utc3['timestamp'] = pd.to_datetime(df_utc3['timestamp'], errors='coerce')
df_utc3['timestamp'] = df_utc3['timestamp'].dt.tz_localize(None)

In [169]:
# Merge both subsets
df_merged = pd.concat([df_utc3, df_utc2], ignore_index=True)

# Optional: sort chronologically
df_merged = df_merged.sort_values(by='timestamp').reset_index(drop=True)

In [173]:
assert len(df) == len(df_merged), "Row count mismatch after splitting and merging!"
print("Original row count:", len(df))
print("Merged row count:  ", len(df_merged))

Original row count: 95490
Merged row count:   95490


In [174]:
df_merged.to_parquet('card_log_0.parquet', index=False)

In [175]:
# Combined into function to process all the files at once
import pandas as pd
import os

def process_file(filename):
    try:
        print(f"Processing {filename} ...")
        df = pd.read_csv(filename, delimiter=' ', names=["date", "timezone", "device_id", "ticket_info", "ticket_id"])

        # Clean and combine timestamp strings
        df['timestamp_raw'] = df['date'].astype(str).str.strip('[') + df['timezone'].astype(str).str.strip(']')

        # Split by timezone offsets using regex on raw string
        df_utc3 = df[df['timestamp_raw'].str.contains(r'\+0300$', regex=True)].copy()
        df_utc2 = df[df['timestamp_raw'].str.contains(r'\+0200$', regex=True)].copy()

        # Parse timestamps
        df_utc3['timestamp'] = pd.to_datetime(df_utc3['timestamp_raw'], format='%d/%b/%Y:%H:%M:%S%z', errors='coerce')
        df_utc2['timestamp'] = pd.to_datetime(df_utc2['timestamp_raw'], format='%d/%b/%Y:%H:%M:%S%z', errors='coerce')

        # Remove timezone info (local naive time)
        df_utc3['timestamp'] = df_utc3['timestamp'].dt.tz_localize(None)
        df_utc2['timestamp'] = df_utc2['timestamp'].dt.tz_localize(None)

        # Combine splits
        df_processed = pd.concat([df_utc3, df_utc2], ignore_index=True)

        print(f"  -> {len(df_processed)} rows processed successfully.")

        return df_processed

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

all_dfs = []
total_files = 13  # files card.log.0 to card.log.12 inclusive

for i in range(total_files):
    filename = f'card.log.{i}'
    if not os.path.exists(filename):
        print(f"Warning: {filename} does not exist, skipping.")
        continue

    df_processed = process_file(filename)
    if not df_processed.empty:
        all_dfs.append(df_processed)

if not all_dfs:
    print("No data processed. Exiting.")
else:
    df_final = pd.concat(all_dfs, ignore_index=True).sort_values('timestamp').reset_index(drop=True)
    print(f"Combined DataFrame has {len(df_final)} rows.")

    output_file = 'card_all_combined.parquet'
    df_final.to_parquet(output_file, index=False, compression='snappy')
    print(f"Data saved to {output_file}.")

Processing card.log.0 ...
  -> 95490 rows processed successfully.
Processing card.log.1 ...
  -> 107301 rows processed successfully.
Processing card.log.2 ...
  -> 243449 rows processed successfully.
Processing card.log.3 ...
  -> 222553 rows processed successfully.
Processing card.log.4 ...
  -> 138620 rows processed successfully.
Processing card.log.5 ...
  -> 86765 rows processed successfully.
Processing card.log.6 ...
  -> 63678 rows processed successfully.
Processing card.log.7 ...
  -> 60960 rows processed successfully.
Processing card.log.8 ...
  -> 43602 rows processed successfully.
Processing card.log.9 ...
  -> 32556 rows processed successfully.
Processing card.log.10 ...
  -> 41538 rows processed successfully.
Processing card.log.11 ...
  -> 8254 rows processed successfully.
Processing card.log.12 ...
  -> 67592 rows processed successfully.
Combined DataFrame has 1212358 rows.
Data saved to card_all_combined.parquet.
